# First Exercise for Pattern Recognition

**Goal:** Implement a KNN (K nearest neighbour) classification algorithm from scratch

## Overview of the approach
1. Read the data from the MNIST Dataset
2. Write KNN Classification Algorithm
4. Run the classification function over the train set to parametrise the mode
5. Test the outcome with the test set

Import needed packages

In [18]:
import numpy
import time
from scipy import spatial


Define Reading function for file paths

In [2]:
def readMnistData(filePath):
    return numpy.genfromtxt(filePath, delimiter=",")

In [3]:
def getClassification(mnistDataEntry: numpy.ndarray):
    return mnistDataEntry[0]

def getMnistData(mnistDataEntry: numpy.ndarray):
    return numpy.delete(mnistDataEntry, [0])

## Euclidean Distance

In [4]:
def euclideanDistance(vectorA: numpy.ndarray, vectorB: numpy.ndarray):
    return numpy.sum(numpy.sqrt(numpy.power(vectorA - vectorB, 2)))

## Manhattan Distance

In [5]:
def manhattanDistance(vectorA: numpy.ndarray, vectorB: numpy.ndarray):
    return numpy.sum(numpy.abs(vectorA - vectorB))

Read the train file

In [6]:
trainData = readMnistData("train.csv")
testData = readMnistData("test.csv")

First try to brute force one classification

## Brute force Iterative
Here follows the first try with just a brute force approach with no optimizations. The programm runs extremely slow. I would not recommend to let it run through.

In [26]:
def bruteforce1nn(toClassify: numpy.ndarray, trainSet: numpy.ndarray):
    withoutClassification = getMnistData(toClassify)
    smallestDistance = 100000
    bestCurrentClassifier = -1
    for currentIndex in range(0, len(trainSet)):
        currentTrainData = testData[currentIndex]
        currentDistance = euclideanDistance(getMnistData(currentTrainData), withoutClassification)
        if smallestDistance > currentDistance:
            smallestDistance = currentDistance
            bestCurrentClassifier = getClassification(currentTrainData)
    return bestCurrentClassifier


def checkClassification(classification, toClassify: numpy.ndarray):
    return classification == getClassification(toClassify)

In [33]:
def getTimeApproxPerClassification(functionToTime, trainDataSetToUse):
    start = time.time()
    for index, currentDataSet in enumerate(testData):
        functionToTime(currentDataSet, trainDataSetToUse)
        print(index)
    end = time.time()
    print(end - start)
    return (end - start)/100

In [ ]:
getTimeApproxPerClassification(bruteforce1nn, testData)

In [ ]:
correct = 0
false = 0
print("Total number of Test cases: "+ str(len(testData)))

for index, currentDataSet in enumerate(testData):
    classifiedAs = bruteforce1nn(currentDataSet, trainData)
    if classifiedAs == getClassification(currentDataSet):
        correct += 1
    else:
        false += 1
    print(str(index) + ": c: " + str(correct) + " f: " + str(false))

accuracy = correct / len(testData)
print("Correct Classified: " + str(correct))
print("False Classified: " + str(false))

print("Accuracy: " + str(accuracy))

## Next Approach: Don't calculate iteratively but use pre build optimized spacial package.
As the above appproach would take ages to run through all the 50'000 entries I decided to move to the scipy.spacial.distance functionality. It is quite optimized and thus takes less time all in all.

In [16]:
def getDistanceMatrix(testData, trainData, metric):
    return  spatial.distance.cdist(getOnlyData(testData), getOnlyData(trainData), metric=metric)

def getMostOccurringElement(row: numpy.ndarray):
    values, counts = numpy.unique(row, return_counts=True)
    return values[numpy.argmax(counts)]


def getAllClassifications(dataMatrix: numpy.matrix):
    return numpy.squeeze(numpy.asarray(dataMatrix[:,0]))

def getOnlyData(dataMatrix: numpy.matrix):
    return numpy.asarray(dataMatrix[:,1:])


def getKNearestNeightbourIndices(distanceMatrix: numpy.ndarray, k):
    kNearestNeighbours = numpy.argpartition(distanceMatrix,k)[:,:k]
    return numpy.squeeze(numpy.asarray(kNearestNeighbours))

def performKNearestNeighbour(distances, trainDataToUse, testDataToUse, k):
    nearestNeighbourIndices = getKNearestNeightbourIndices(distances, k)
    allPredictions = getAllClassifications(trainDataToUse)[nearestNeighbourIndices]
    if len(numpy.shape(allPredictions)) > 1:
        predictions = numpy.apply_along_axis(lambda v: getMostOccurringElement(v), axis=1, arr=allPredictions)
        return predictions
    else:
        return allPredictions

def getCorrectness(madeClassifications, trueClassifications):
    subtractedClassifications = numpy.subtract(trueClassifications, madeClassifications)
    numberOfFalse = numpy.count_nonzero(subtractedClassifications)
    return 1 - numberOfFalse / len(trueClassifications)

In [11]:
euclideanDistanceMatrix = getDistanceMatrix(testData, trainData, 'euclid')


In [9]:
manhattanDistanceMatrix = getDistanceMatrix(testData, trainData, 'cityblock')

In [17]:
def performNKNNClassifications(ks: numpy.ndarray, distanceMatrix, trainData, testData):
    for k in ks:
        predictedClass = performKNearestNeighbour(distanceMatrix, trainData, testData, k)
        accuracy = getCorrectness(predictedClass, getAllClassifications(testData))
        print(f"| {k} | {round(accuracy, 3)}|")

ks = numpy.array([1, 3, 5, 10, 15])
print("Accuracy for Euclid")
performNKNNClassifications(ks, euclideanDistanceMatrix, trainData, testData)
print("And for Manhattan")
performNKNNClassifications(ks, manhattanDistanceMatrix, trainData, testData)

Accuracy for Euclid
| 1 | 0.965|
| 3 | 0.964|
| 5 | 0.964|
| 10 | 0.96|
| 15 | 0.956|
And for Manhattan
| 1 | 0.957|
| 3 | 0.957|
| 5 | 0.957|
| 10 | 0.952|
| 15 | 0.946|


# Accuracies for euclidean Distance
The Accuracies stay pretty much the same with a slight decline with higher ks

| k   | accuracy |
|-----|----------|
| 1   | 0.964    |
| 3   | 0.964    |
| 5   | 0.963    |
| 10  | 0.960    |
| 15  | 0.955    |

# Accuracies for Manhattan Distance
They behave quite like the euclidean

| k   | accuracy |
|-----|----------|
| 1 | 0.957|
| 3 | 0.957|
| 5 | 0.957|
| 10 | 0.952|
